Showcase for Large Displacement Optical Flow

In [30]:
#  Initialize and import modules
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
import matplotlib.animation as animation
import pyidi
# properties constants and paths
# path = "C:/Users/thijs/Documents/GitHub/pyidi/data/"
# filename = 'synthetic_data_thijs_without_rotation'

filename = 'Single_line_15_S01'
path = 'D:/HSC/'+filename+'/'+filename+'.cihx'

In [32]:
# video = pyidi.pyIDI(path + filename +'.cih')
video = pyidi.pyIDI(path)
mraw = video.mraw
image_width = video.info['Image Width']
image_height = video.info['Image Height']
total_frame = video.info['Total Frame']

c:\Users\thijs\anaconda3\lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


In [33]:
#  Simulate MRAW
%matplotlib qt
def play_video(frame_range, mraw, interval=30, d = None):
    fig, ax = plt.subplots()
    im = ax.imshow(mraw[frame_range[0]], cmap='gray', vmin=0, vmax=2**16-1)
    if d is not None:
        ps, = ax.plot(d[0][0], d[0][1], 'r.', markersize=3)
    ax.set_xlim(0, image_width)
    ax.set_ylim(image_height,0)
    ax.set_aspect('equal')
    text = ax.text(0.95, 0.05, '', transform=ax.transAxes, color='black', ha='right', va='bottom')
    def update(it):
        im.set_data(mraw[it])
        text.set_text(f'Frame {it}')
        if d is not None:
            ps.set_data(d[it][0], d[it][1])
        return im, text

    ani = animation.FuncAnimation(fig, update, frames=frame_range, interval=interval) #
    plt.show()
    return ani
mraw_range = (0, 11)
ani = play_video(range(mraw_range[0], mraw_range[1]), mraw, interval=1) #fig_ax = (fig, ax)


In [36]:
still_image = mraw[mraw_range[0]]
gy, gx = np.gradient(still_image, edge_order=2)
point = np.argmax(np.linalg.norm(np.dstack((gx, gy)), axis=2))
# point = np.argmax(gx)
point = np.unravel_index(point, still_image.shape)
point = (point[0], point[1])


pix_i = np.array([116, 117,], dtype=np.int64)
pix_j = np.array([373, 375,], dtype=np.int64)
points = np.stack((pix_i, pix_j), axis=-1)
# fig, ax = plt.subplots()
# ax.imshow(still_image, cmap='gray')
# ax.plot(point[1], point[0], 'r.')

In [37]:
video.set_method(method='sof')
video.method.configure(pixel_shift = True, mraw_range=mraw_range, subset_size=1)
# video.set_points(points = [point])
video.set_points(points = points)
video.show_points()

In [ ]:
# video.method.gradient_magnitude

In [40]:
d = video.get_displacements()
# video.method.in_range_points
# d = d[0]
d

100%|██████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 528.50it/s]


array([[[ 1.27282232, -0.48235668],
        [ 0.72712891,  0.10340893],
        [ 0.71799526,  0.10687027],
        ...,
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.98060768, -0.73021317],
        [ 0.87669861, -0.90818316],
        [ 0.91034156, -0.93323551],
        ...,
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]])

In [39]:
pix_i = point[1] + d[:,1]
pix_j = point[0] + d[:,0]
td = np.array([pix_i, pix_j]).T
ani = play_video(range(mraw_range[0], mraw_range[1]), mraw, interval=1, d=td)
mraw_range

(0, 11)

C:\Users\thijs\AppData\Local\Temp\ipykernel_11400\3048367314.py:16: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ps.set_data(d[it][0], d[it][1])
